# Fine tuning T5 with Layer

[![Open In Layer](https://uploads-ssl.webflow.com/6090530ad3595f001f4f9084/623dc0c25d2cf70aa28210bc_layer_badge.svg)](https://development.layer.co/layer/t5-fine-tuning-with-layer?tab=assets) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1fppz3WBPnX-XUO4ucRe1cv6CUXNCrBAg?usp=sharing)

A T5 is an encoder-decoder model. It converts all NLP problems like language translation, summarization, text generation, question-answering, to a text-to-text task.

We are going to fine tune a pretrained T5 Model from 🤗 and train it to translate English to SQL.

# Install Requirements

In [ ]:
!pip install layer-sdk --upgrade -q
!pip install sentencepiece -q
!pip install transformers -q

In [3]:
from layer.decorators.assertions import assert_true, assert_valid_values, assert_not_null, assert_unique
from layer.decorators import dataset, model,resources, pip_requirements,fabric
from layer.client import Dataset
import layer
import torch
import random
import re
import pandas as pd
import numpy as np
import os

# Getting Started with Layer

Layer is an MLOps platform which advances ML pipelines with remote computation and tracking.

## Login to Layer

Let's login to Layer first.

In [ ]:
layer.login("https://development.layer.co")

## Initialize Layer Project
Now we are ready to init our project. Layer Project is basically an ML Repo hosted on Layer where you can store your datasets, models, metrics

In [5]:
layer.init("t5-fine-tuning-with-layer")

Project(name='t5-fine-tuning-with-layer', raw_datasets=[], derived_datasets=[], featuresets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', organization=Organization(id=UUID('d7325da3-0646-4fa6-855d-8d19eece8b79'), name='layer'), _id=UUID('4426cf2d-bc6d-487f-902a-1b106e04da58'), functions=[])

Your project is ready. Find your project here:

https://development.layer.co

# Dataset Generation
Unlike language to language translation datasets, we can build custom English to SQL translation pairs programmatically with the help of some templates.

In [6]:
templates = [
              ["[prop1] of [nns]","SELECT [prop1] FROM [nns]"],
              ["[agg] [prop1] for each [breakdown]","SELECT [agg]([prop1]) , [breakdown] FROM [prop1] GROUP BY [breakdown]"],
              ["[prop1] of [nns] by [breakdown]","SELECT [prop1] , [breakdown] FROM [nns] GROUP BY [breakdown]"],
              ["[prop1] of [nns] in [location] by [breakdown]","SELECT [prop1] , [breakdown] FROM [nns] WHERE location = '[location]' GROUP BY [breakdown]"],
              ["[nns] having [prop1] between [number1] and [number2]","SELECT name FROM [nns] WHERE [prop1] > [number1] and [prop1] < [number2]"],
              ["[prop] by [breakdown]","SELECT name , [breakdown] FROM [prop] GROUP BY [breakdown]"],
              ["[agg] of [prop1] of [nn]","SELECT [agg]([prop1]) FROM [nn]"],
              ["[prop1] of [nns] before [year]","SELECT [prop1] FROM [nns] WHERE date < [year]"],
              ["[prop1] of [nns] after [year] in [location]","SELECT [prop1] FROM [nns] WHERE date > [year] AND location='[location]'"],
              ["[nns] [verb] after [year] in [location]","SELECT name FROM [nns] WHERE location = '[location]' AND date > [year]"],
              ["[nns] having [prop1] between [number1] and [number2] by [breakdown]","SELECT name , [breakdown] FROM [nns] WHERE [prop1] < [number1] AND [prop1] > [number2] GROUP BY [breakdown]"],
              ["[nns] with a [prop1] of maximum [number1] by their [breakdown]","SELECT name , [breakdown] FROM [nns] WHERE [prop1] <= [number1] GROUP BY [breakdown]"],
              ["[prop1] and [prop2] of [nns] since [year]","SELECT [prop1] , [prop2] FROM [nns] WHERE date > [year]"],
              ["[nns] which have both [prop1] and [prop2]","SELECT name FROM [nns] WHERE [prop1] IS true AND [prop2] IS true"],
              ["Top [number1] [nns] by [prop1]","SELECT name FROM [nns] ORDER BY [prop1] DESC LIMIT [number1]"]
]
template = random.choice(templates)
print("Sample Query Template  :", template[0])
print("SQL Translation        :", template[1])

Sample Query Template  : Top [number1] [nns] by [prop1]
SQL Translation        : SELECT name FROM [nns] ORDER BY [prop1] DESC LIMIT [number1]


In [7]:
objects = ["countries","wines","wineries","tasters", "provinces","grapes","cities","bottles","deliveries"]
object_single = ["country","wine","winery","taster", "province","grape","city","bottle", "delivery"]
properties = ["points","price","taste","title","texture","age","duration","acidity","flavor","level"]
aggs = [["average","avg"], ["total","sum"],["count","count"], ["minimum","min"], ["maximum","max"]]
breakdowns = ["quality","price","province","country","point", "variety","flavor","age"]
locations = ["Italy","US","Portugal","Spain","Chile","Turkey","Canada"]
verbs = ["produced","bottled"]

regex = r"\[([a-z0-9]*)\]"
number_of_samples = 2500

@dataset("english_sql_translations")
def build_dataset():
    rows = []
    for index in range(0,number_of_samples):
        template = random.choice(templates)
        nl = template[0]
        sql = template[1]

        matches = re.finditer(regex, nl, re.MULTILINE)

        for matchNum, match in enumerate(matches, start=1):
            key = match.group()
            prop = None
            prop_sql = None
            if key.startswith("[prop"):
                prop = random.choice(properties)
                prop_sql = prop.replace(" ","_").lower()
            if key in ["[nns]"]:
                prop = random.choice(objects)
                prop_sql = prop
            if key in ["[nn]"]:
                prop = random.choice(object_single)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[breakdown]":
                prop = random.choice(breakdowns)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[verb]":
                prop = random.choice(verbs)
                prop_sql = prop.replace(" ","_").lower()
            if key == "[agg]":
                aggregation = random.choice(aggs)
                prop = aggregation[0]
                prop_sql = aggregation[1]
            if key == "[location]":
                prop = random.choice(locations)
                prop_sql = prop
            if key.startswith("[number"):
                prop = str(random.randint(1,1000))
                prop_sql = prop
            if key.startswith("[year"):
                prop = str(random.randint(1950,2022))
                prop_sql = prop
            

            if prop is not None:
                nl = nl.replace(key,prop)
                sql = sql.replace(key,prop_sql)
        
        prefix = random.randint(1,20)
        if prefix == 1:
            nl = "Show me "+nl
        elif prefix == 2:
            nl = "List "+nl
        elif prefix == 3:
            nl = "List of "+nl
        elif prefix == 4:
            nl = "Find "+nl
        rows.append([nl,sql])

    df = pd.DataFrame(rows, columns=["query", "sql"])
    return df

## Register dataset to Layer

In the above cell, we have used a special decorator called `@dataset` which tells Layer that our function creates dataset. Now we are going to pass this function to Layer to be run on Layer infra and register the built dataset under our project.

In [ ]:
layer.run([build_dataset])

## Create Data Loader

In [10]:
class EnglishToSQLDataSet(Dataset):

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

    self.data["query"] = "translate English to SQL: "+self.data["query"]
    self.data["sql"] = "<pad>" + self.data["sql"] + "</s>"

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

# Fine Tune T5

Our dataset is ready and registered to Layer. Now we are going to develop the fine tuning logic, decorate the function with `@model` and pass it to Layer so that it can be run on Layer infra and registered under our project

In [11]:
def train(epoch, tokenizer, model, device, loader, optimizer):
  import torch
  
  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%50==0:
      print(str(epoch), str(_), str(loss), flush=True)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Here we use 3 seperate Layer decorators:
- [`@model`](https://docs.development.layer.co/docs/sdk-library/model-decorator): Tells Layer that this function trains an ML model
- [`@fabric`](https://docs.development.layer.co/docs/sdk-library/fabric-decorator): Tells Layer the computation resources (cpu, gpu etc.) needed to train the model. Here is a list of the [available fabrics](https://docs.development.layer.co/docs/reference/fabrics) you can use.
- [`@pip_requirements`](https://docs.development.layer.co/docs/sdk-library/pip-requirements-decorator): Tells the pypi libraries needed to train the model.

In [16]:
@model("t5-tokenizer")
@fabric("f-medium")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_tokenizer():
  from transformers import T5Tokenizer
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
  return tokenizer

@model("t5-english-to-sql")
@fabric("f-medium")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_model():
  from transformers import T5Tokenizer, T5ForConditionalGeneration
  from torch import cuda
  from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
  import torch.nn.functional as F
  import torch

  # # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # Setting up the device for GPU usage
  device = 'cuda' if cuda.is_available() else 'cpu'

  print(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # Load tokenizer
  tokenizer = layer.get_model("t5-tokenizer").get_train()
  
  # Load pretrained model from Layer
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model.to(device)

  print(f"[Data]: Reading data...\n")
  
  dataframe = layer.get_dataset("english_sql_translations").to_pandas()
  source_text = "query"
  target_text = "sql"

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]

  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  print(f"FULL Dataset: {dataframe.shape}")
  print(f"TRAIN Dataset: {train_dataset.shape}")
  print(f"TEST Dataset: {val_dataset.shape}\n")

  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = EnglishToSQLDataSet(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = EnglishToSQLDataSet(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)

  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  print(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)

  print(f"[Saving Model]...\n")

  return model


In [ ]:
model_params={
    "MODEL":"t5-small",            
    "TRAIN_BATCH_SIZE":8,          
    "VALID_BATCH_SIZE":8,          
    "TRAIN_EPOCHS":3,              
    "VAL_EPOCHS":3,                
    "LEARNING_RATE":1e-4,          
    "MAX_SOURCE_TEXT_LENGTH":75,   
    "MAX_TARGET_TEXT_LENGTH":75,
    "SEED": 33
}

# # You can train your model locally by just calling the function to debug your code.
# build_tokenizer()
# build_model()

# # Once you are ready, you can push your model training function to Layer to be trained.
layer.run([build_tokenizer, build_model], debug=True)